### LLM Model Demo

In [ ]:
# Installed LangChain 
#%pip install langchain langchain-community faiss-cpu langchain-chroma langchain-openai langchain_text_splitters langgraph beautifulsoup4 langchain-huggingface

In [ ]:
#%pip install langchain_huggingface

In [3]:
import langchain
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
import datetime
import os
from langchain_community.document_loaders import PyPDFDirectoryLoader # type: ignore
from langchain.embeddings import HuggingFaceEmbeddings # type: ignore
from langchain_community.vectorstores import FAISS # type: ignore
from langchain.chains import RetrievalQA # type: ignore
from langchain_ollama.llms import OllamaLLM # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
from langchain_core.messages import HumanMessage, SystemMessage # type: ignore
from langchain_core.prompts import ChatPromptTemplate # type: ignore

from langchain.cache import InMemoryCache

In [4]:
os.environ["AZURE_OPENAI_API_KEY"] = "ce53c5fce80c4503927244333e40634c"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://wdc-chat-llm.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"
os.environ["OPENAI_API_VERSION"] = "2024-06-01"


In [5]:
llm = AzureChatOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version = os.environ["OPENAI_API_VERSION"],
    temperature= 0.3
)
langchain.llm_cache = InMemoryCache() # type: ignore
parser = StrOutputParser()
#embeddings = HuggingFaceEmbeddings() #model="sentence-transformers/all-mpnet-base-v2"

In [6]:
messages = [
    SystemMessage(content="Translate the following from English into Bengali"),
    HumanMessage(content="hi!"),
]

In [7]:
prompt = "How are you doing?"

In [8]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [9]:
result = llm.invoke(messages)
parser.invoke(result)

APIConnectionError: Connection error.

In [ ]:
chain = prompt_template | llm | parser
chain.invoke({"language": "bengali", "text": "how are you doing"})

In [10]:
#%pip install langchain_ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(
    model="llama3.2",
    top_k=3
    )

chain = prompt | model

chain.invoke({"question": "How far Earth from Sun?"})

"To calculate the average distance of Earth from the Sun, we need to consider a few factors.\n\nStep 1: The average distance between the Earth and the Sun is called an astronomical unit (AU). One AU is equal to the average distance between the Earth and the Sun, which is approximately 93 million miles or 149.6 million kilometers.\n\nStep 2: To get more precise measurements, we can use the elliptical shape of the Earth's orbit around the Sun. At its closest point (perihelion), the distance is about 91.4 million miles or 147 million kilometers. At its farthest point (aphelion), the distance is about 94.5 million miles or 152.1 million kilometers.\n\nStep 3: The Earth's orbit is not a perfect circle, so we need to use an average value that takes into account both perihelion and aphelion distances. A commonly used average distance is 93.8 million miles (151.4 million kilometers) from the Sun.\n\nSo, to summarize, the average distance of Earth from the Sun is approximately 93.8 million mile

In [ ]:
%pip install langchain faiss-cpu unstructured[pdf]

In [ ]:
# filepath: rag_implementation.py


# Load documents from a local folder
def load_documents_from_folder(folder_path):
    #loader = DirectoryLoader(folder_path,glob="**/*.pdf",show_progress=True)
    loader = PyPDFDirectoryLoader(folder_path)
    documents = loader.load()
    return documents

# Embed and vectorize documents using FAISS
def embed_and_vectorize_documents(documents):
    embeddings = OllamaEmbeddings(model="llama3.2")
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

# Implement RAG
def implement_rag(vector_store, query):
    llm  = OllamaLLM(
    model="llama3.2"
    )
    retriever = vector_store.as_retriever()
    rag_chain = RetrievalQA(llm=llm, retriever=retriever)
    result = rag_chain.run(query)
    return result

messages = [
    SystemMessage(content="As Technical Recruiter Evaluate the following resumes"),
    HumanMessage(content="Hello Recruiter"),
]

system_template = "Translate the following into {language}:"

"""prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
) """

if __name__ == "__main__":
    folder_path = r"C:\Users\HSASS\OneDrive - Wipro\Desktop\Temp2\Resume"
    query = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")])

    # Load documents
    documents = load_documents_from_folder(folder_path)

    # Embed and vectorize documents
    vector_store = embed_and_vectorize_documents(documents)

    # Implement RAG and get the result
    result = implement_rag(vector_store, query)
    print(result)
    